In [1]:
file_dir = './'
frame_dir = file_dir + 'frames/'
label_dir = file_dir + 'labels/'

In [2]:
import numpy as np
from scipy import misc
import glob
import os, os.path

In [3]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import time
from scipy.misc import imread
from scipy.misc import imresize
import matplotlib.image as mpimg
from scipy.ndimage import filters
import urllib
from numpy import random

import tensorflow as tf

In [12]:
def load_data(batch_size):
    #train_x_head = []
    train_x_lhand= []
    #train_x_rhand= []
    train_y = []
    #frames train house 2
    random.seed(9001)
    place = ['house/', 'lab/', 'office/'][random.randint(0, 2)]
    if place is 'lab/':
        num = random.randint(1, 4)
        
        fdir = frame_dir+'train/'+place+str(num)+'/'
        ldir = label_dir+place
        #load labels
        FA_left = np.eye(2)[np.load(ldir+'FA_left'+str(num)+'.npy').astype(int)]
        

        ges_left = np.eye(13)[np.load(ldir+'ges_left'+str(num)+'.npy').astype(int)]
        

        obj_left = np.eye(24)[np.load(ldir+'obj_left'+str(num)+'.npy').astype(int)]
        

        train_y.extend(concatenate((FA_left, ges_left, obj_left), axis=1))
        
        
        file_count = len(os.listdir(fdir+'head/'))
        index = random.choice(list(range(file_count)), batch_size)
        train_y = [train_y[i] for i in index]
        
        #load frames
        for i in index+1:
            
            train_x_lhand.append(misc.imresize(misc.imread(fdir+'Lhand/Image'+str(i)+'.png'), [135,240]))
 
    else:
        num = random.randint(1, 3)
        
        fdir = frame_dir+'train/'+place+str(num)+'/'
        ldir = label_dir+place
        #load labels
        FA_left = np.eye(2)[np.load(ldir+'FA_left'+str(num)+'.npy').astype(int)]
        #FA_right = np.eye(2)[np.load(ldir+'FA_right'+str(num)+'.npy').astype(int)]

        ges_left = np.eye(13)[np.load(ldir+'ges_left'+str(num)+'.npy').astype(int)]
        #ges_right = np.eye(13)[np.load(ldir+'ges_right'+str(num)+'.npy').astype(int)]

        obj_left = np.eye(24)[np.load(ldir+'obj_left'+str(num)+'.npy').astype(int)]

        train_y.extend(concatenate((FA_left, ges_left, obj_left), axis=1))
        
        
        file_count = len(os.listdir(fdir+'head/'))
        index = random.choice(list(range(file_count)), batch_size)
        train_y = [train_y[i] for i in index]
        
        #load frames
        for i in index+1:
            
            train_x_lhand.append(misc.imresize(misc.imread(fdir+'Lhand/Image'+str(i)+'.png'), [135,240]))
 
                            

    return [train_x_lhand, train_y]

In [13]:
def test_data():
    test_x_dir = []
    #train_x_rhand= []
    test_y = []
    #frames train house 2
    for place in ['house/', 'lab/', 'office/']:
        if place is 'lab/':
            for num in range(1,5):
                
                fdir = frame_dir+'test/'+place+str(num)+'/'
                ldir = label_dir+place
                #load labels
                FA_left = np.eye(2)[np.load(ldir+'FA_left'+str(3+num)+'.npy').astype(int)]
                ges_left = np.eye(13)[np.load(ldir+'ges_left'+str(3+num)+'.npy').astype(int)]
                obj_left = np.eye(24)[np.load(ldir+'obj_left'+str(3+num)+'.npy').astype(int)]
                test_y.extend(concatenate((FA_left, ges_left, obj_left), axis=1))

                file_count = len(os.listdir(fdir+'head/'))

                #load frames
                for i in range(file_count):
                    test_x_dir.append(fdir+'Lhand/Image'+str(i+1)+'.png')
               
        else:
             for num in range(1,4):

                fdir = frame_dir+'test/'+place+str(num)+'/'
                ldir = label_dir+place
                #load labels
                FA_left = np.eye(2)[np.load(ldir+'FA_left'+str(3+num)+'.npy').astype(int)]
                ges_left = np.eye(13)[np.load(ldir+'ges_left'+str(3+num)+'.npy').astype(int)]
                obj_left = np.eye(24)[np.load(ldir+'obj_left'+str(3+num)+'.npy').astype(int)]
                test_y.extend(concatenate((FA_left, ges_left, obj_left), axis=1))

                file_count = len(os.listdir(fdir+'head/'))

                #load frames
                for i in range(file_count):
                    test_x_dir.append(fdir+'Lhand/Image'+str(i+1)+'.png')
    return [test_x_dir, test_y]

In [14]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#xt_head, xt_lhand, xt_rhand, yt = load_data()

In [17]:

# hyperparameters
learning_rate = 0.0001
training_iters = 1000
batch_size = 20
display_step = 100

# other parameters
w_input = 135
h_input = 240
d_input = 3
n_classes = 2+13+24
dropout = 0.8 

# placeholders
x = tf.placeholder(tf.float32, [None, w_input, h_input, d_input])
#x_lhand = tf.placeholder(tf.float32, [batch_size, w_input, h_input, d_input])
#x_rhand = tf.placeholder(tf.float32, [batch_size, w_input, h_input, d_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# convolution
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding='SAME'),b), name=name)

# max pooling
def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)

# batch norm
def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

# network 
def alex_net(_X, _weights, _biases, _dropout):

    # conv 1
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    pool1 = max_pool('pool1', conv1, k=2)
    norm1 = norm('norm1', pool1, lsize=4)
    norm1 = tf.nn.dropout(norm1, _dropout)

    # conv 2
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    pool2 = max_pool('pool2', conv2, k=2)
    norm2 = norm('norm2', pool2, lsize=4)
    norm2 = tf.nn.dropout(norm2, _dropout)

    # conv 3
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    pool3 = max_pool('pool3', conv3, k=2)
    norm3 = norm('norm3', pool3, lsize=4)
    norm3 = tf.nn.dropout(norm3, _dropout)

    # conv 4
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    pool3 = max_pool('pool3', conv3, k=2)
    norm3 = norm('norm3', pool3, lsize=4)
    norm3 = tf.nn.dropout(norm3, _dropout)

#FA
    # fc 1
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1') 
    # fc 2
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation

    # output layer
    out1 = tf.matmul(dense2, _weights['out']) + _biases['out']
    
#ges
    dense1 = tf.reshape(norm3, [-1, _weights['wd12'].get_shape().as_list()[0]]) 
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd12']) + _biases['bd12'], name='fc12') 
    # fc 2
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd22']) + _biases['bd22'], name='fc22') # Relu activation

    # output layer
    out2 = tf.matmul(dense2, _weights['out2']) + _biases['out2']
#obj
    dense1 = tf.reshape(norm3, [-1, _weights['wd13'].get_shape().as_list()[0]]) 
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd13']) + _biases['bd13'], name='fc13') 
    # fc 2
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd23']) + _biases['bd23'], name='fc23') # Relu activation

    # output layer
    out3 = tf.matmul(dense2, _weights['out3']) + _biases['out3']
    
    out = tf.concat([out1, out2, out3], 1)
    
    return out

# weights and biases for the model
weights_head = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 3, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wc3': tf.Variable(tf.random_normal([5, 5, 64, 128])),
    'wd1': tf.Variable(tf.random_normal([65280, 256])),
    'wd2': tf.Variable(tf.random_normal([256, 128])),
    'out': tf.Variable(tf.random_normal([128, 2])), 
    
    'wd12': tf.Variable(tf.random_normal([65280, 256])),
    'wd22': tf.Variable(tf.random_normal([256, 128])),
    'out2': tf.Variable(tf.random_normal([128, 13])),
    
    'wd13': tf.Variable(tf.random_normal([65280, 256])),
    'wd23': tf.Variable(tf.random_normal([256, 128])),
    'out3': tf.Variable(tf.random_normal([128, 24]))
}
biases_head = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    
    'bd1': tf.Variable(tf.random_normal([256])),
    'bd2': tf.Variable(tf.random_normal([128])),
    'out': tf.Variable(tf.random_normal([2])),
    
    'bd12': tf.Variable(tf.random_normal([256])),
    'bd22': tf.Variable(tf.random_normal([128])),
    'out2': tf.Variable(tf.random_normal([13])),
    
    'bd13': tf.Variable(tf.random_normal([256])),
    'bd23': tf.Variable(tf.random_normal([128])),
    'out3': tf.Variable(tf.random_normal([24]))
}


# network
pred = alex_net(x, weights_head, biases_head, keep_prob)


def pred_net(x1, x2, _weights, _biases, _dropout):
    x = tf.concat([x1, x2], axis = 0)
    print(x)
    # fc 1
    dense1 = tf.reshape(x, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1') 
    # fc 2
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation

    # output layer
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    return out



# loss function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred[:,0:2], labels = y[:,0:2])) +\
tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred[:,2:15], labels = y[:,2:15]))+\
tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred[:,15:], labels = y[:,15:]))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# prediction
correct_fa = tf.equal(tf.argmax(pred[:,0:2],1), tf.argmax(y[:,0:2],1))
correct_ges = tf.equal(tf.argmax(pred[:,2:15],1), tf.argmax(y[:,2:15],1))
correct_obj = tf.equal(tf.argmax(pred[:,15:],1), tf.argmax(y[:,15:],1))
#correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy_fa = tf.reduce_mean(tf.cast(correct_fa, tf.float32))
accuracy_ges = tf.reduce_mean(tf.cast(correct_ges, tf.float32))
accuracy_obj = tf.reduce_mean(tf.cast(correct_obj, tf.float32))

#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# initial
init = tf.global_variables_initializer()

# start training

with tf.Session(config=config) as sess:
    sess.run(init)
    step = 1
    print("start training")
    #batch_size = 10
    # Keep training until reach max iterations
    while step < training_iters:
        xt, yt = load_data(batch_size)
        
        sess.run(optimizer, feed_dict={x:xt, y: yt, keep_prob: dropout})
        if step % display_step == 0:
            # accuracy
            acc_fa, acc_ges, acc_obj = sess.run([accuracy_fa, accuracy_ges, accuracy_obj], feed_dict={x: xt, y: yt, keep_prob: 1.})
            # loss
            loss = sess.run(cost, feed_dict={x: xt,y: yt, keep_prob: 1.})
            print("Iter " + str(step) + ", Minibatch Loss= " + "{:.2f}".format(loss)+ ", Training Accuracy FA= " 
                  + "{:.5f}".format(acc_fa)+ ", Training Accuracy GES= " 
                  + "{:.5f}".format(acc_ges)+ ", Training Accuracy OBJ= " 
                  + "{:.5f}".format(acc_obj))
        step += 1
    print("Optimization Finished!")
    # test acc
    xtest_dir, ytest = test_data()
    test_size = 20
    all_acc = [0, 0, 0]
    for i in range(int(len(xtest_dir)/test_size)):
        #read imgs from that
        xtest = []
        for j in range(test_size):
            xtest.append(misc.imresize(misc.imread(xtest_dir[i*test_size+j]), [135,240]))

        fromx = i*test_size
        tox = (i+1)*test_size
        all_acc += sess.run([accuracy_fa, accuracy_ges, accuracy_obj], feed_dict={x: xtest, y: ytest[fromx:tox], keep_prob: 1.})

    avg_acc = [all_acc[0]/(int(len(xtest_dir)/test_size)), all_acc[1]/(int(len(xtest_dir)/test_size)),all_acc[2]/(int(len(xtest_dir)/test_size))]
    print("Testing Accuracy FA:"+"{:.5f}".format(avg_acc[0])+"Testing Accuracy GES:"+"{:.5f}".format(avg_acc[1])+"Testing Accuracy FA:"+"{:.5f}".format(avg_acc[2]))

start training
Iter 100, Minibatch Loss= 127651.03, Training Accuracy FA= 0.75000, Training Accuracy GES= 0.30000, Training Accuracy OBJ= 0.10000
Iter 200, Minibatch Loss= 74700.05, Training Accuracy FA= 0.85000, Training Accuracy GES= 0.30000, Training Accuracy OBJ= 0.35000
Iter 300, Minibatch Loss= 56194.27, Training Accuracy FA= 0.85000, Training Accuracy GES= 0.45000, Training Accuracy OBJ= 0.45000
Iter 400, Minibatch Loss= 37665.08, Training Accuracy FA= 0.80000, Training Accuracy GES= 0.45000, Training Accuracy OBJ= 0.55000
Iter 500, Minibatch Loss= 30029.28, Training Accuracy FA= 0.70000, Training Accuracy GES= 0.60000, Training Accuracy OBJ= 0.60000
Iter 600, Minibatch Loss= 24170.48, Training Accuracy FA= 0.70000, Training Accuracy GES= 0.60000, Training Accuracy OBJ= 0.65000
Iter 700, Minibatch Loss= 15219.27, Training Accuracy FA= 0.70000, Training Accuracy GES= 0.80000, Training Accuracy OBJ= 0.85000
Iter 800, Minibatch Loss= 12286.51, Training Accuracy FA= 0.75000, Trainin

TypeError: unsupported operand type(s) for /: 'list' and 'int'